# Importing Needed Libraries and Config

In [1]:
# to import configured paths and other settings
import configparser
# standard libraries for data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# importing config.ini settings
config = configparser.ConfigParser()
config.read("../src/config.ini")

# output path
output = config['paths']['data_path']


# Importing Data

In [3]:
# importing data
test_data = pd.read_csv(output+'test.csv')
train_data = pd.read_csv(output+'train.csv')
samp_sub = pd.read_csv(output+'sample_submission.csv')

In [4]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
samp_sub.head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


Checking the shape. 
Both the test data and train data are nearly the same size.

In [7]:
print(test_data.shape)
print(train_data.shape)

(1459, 80)
(1460, 81)


Checking Nulls 

In [8]:
has_null = []
to_drop_null = []
for col in train_data.columns:
    null_pct = train_data[col].isnull().sum()/train_data.shape[0]
    if null_pct == 0:
        continue
    else:
        if null_pct<0.10:
            has_null.append(col)
        else:
            to_drop_null.append(col)
        

In [9]:
to_drop_null

['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature']

In [10]:
has_null

['MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [11]:
for col in has_null:
    print(col,"\n", train_data[col].value_counts(normalize=True, dropna=False), "\n")

MasVnrType 
 None       0.591781
BrkFace    0.304795
Stone      0.087671
BrkCmn     0.010274
NaN        0.005479
Name: MasVnrType, dtype: float64 

MasVnrArea 
 0.0      0.589726
NaN      0.005479
72.0     0.005479
108.0    0.005479
180.0    0.005479
           ...   
562.0    0.000685
89.0     0.000685
921.0    0.000685
762.0    0.000685
119.0    0.000685
Name: MasVnrArea, Length: 328, dtype: float64 

BsmtQual 
 TA     0.444521
Gd     0.423288
Ex     0.082877
NaN    0.025342
Fa     0.023973
Name: BsmtQual, dtype: float64 

BsmtCond 
 TA     0.897945
Gd     0.044521
Fa     0.030822
NaN    0.025342
Po     0.001370
Name: BsmtCond, dtype: float64 

BsmtExposure 
 No     0.652740
Av     0.151370
Gd     0.091781
Mn     0.078082
NaN    0.026027
Name: BsmtExposure, dtype: float64 

BsmtFinType1 
 Unf    0.294521
GLQ    0.286301
ALQ    0.150685
BLQ    0.101370
Rec    0.091096
LwQ    0.050685
NaN    0.025342
Name: BsmtFinType1, dtype: float64 

BsmtFinType2 
 Unf    0.860274
Rec    0.036986
Lw

In [15]:
fill_dict = {'MasVnrType':"None",
            'MasVnrArea':0,
            'BsmtQual':"None",
            'BsmtCond':"None",
            'BsmtExposure':"None",
            'BsmtFinType1':"None",
            'BsmtFinType2':"None",
            'Electrical':"SBrkr",
            'GarageType':"None",
            'GarageYrBlt':0,
            'GarageFinish':"None",
            'GarageQual':"None",
            'GarageCond':"None",
            'MSZoning': "Unknown",
            'Utilities': "Unknown",
            'Exterior1st': "Other",
            'Exterior2nd': "Other",
            'BsmtFinSF2': "None",
            'KitchenQual': "TA",
            'Functional': "Mod",
            'SaleType': "Other"}

In [16]:
train_data.fillna(fill_dict, inplace=True)
test_data.fillna(fill_dict, inplace=True)

In [17]:
train_data.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [ ]:
train_data.drop(columns=to_drop_null,inplace=True)
test_data.drop(columns=to_drop_null, inplace=True)

In [ ]:
for col in train_data.columns:
    if col not in test_data.columns:
        print(col)

SalePrice


In [ ]:
test_data.isna().sum()

Id               0
MSSubClass       0
MSZoning         4
LotArea          0
Street           0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         1
SaleCondition    0
Length: 74, dtype: int64

In [ ]:
has_null_test = []
to_drop_null_test = []
for col in test_data.columns:
    null_pct = test_data[col].isnull().sum()/test_data.shape[0]
    if null_pct == 0:
        continue
    else:
        if null_pct<0.10:
            has_null_test.append(col)
        else:
            to_drop_null_test.append(col)

In [ ]:
has_null_test

['MSZoning',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageCars',
 'GarageArea',
 'SaleType']

In [ ]:
train_data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
train_data.index = train_data['Id']

In [ ]:
train_data.drop(columns='Id', inplace=True)

In [ ]:
labels

Id
1       208500
2       181500
3       223500
4       140000
5       250000
         ...  
1456    175000
1457    210000
1458    266500
1459    142125
1460    147500
Name: SalePrice, Length: 1460, dtype: int64

In [ ]:
train_data.columns

Index(['MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorc

In [ ]:
train_data.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,2,2008,WD,Normal,208500
2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,0,0,0,0,0,5,2007,WD,Normal,181500
3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,0,0,0,0,0,9,2008,WD,Normal,223500
4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [ ]:
cats = train_data.select_dtypes(include='O')

In [ ]:
nums = train_data.select_dtypes(include='int')

In [ ]:
floats = train_data.select_dtypes(include='float')

In [ ]:
cats = pd.get_dummies(cats)

In [ ]:
cats

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0
5,0,0,0,1,0,0,1,1,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1457,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1458,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [ ]:
def memory_reduction(df):
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

In [ ]:
nums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1460 non-null   int64
 1   LotArea        1460 non-null   int64
 2   OverallQual    1460 non-null   int64
 3   OverallCond    1460 non-null   int64
 4   YearBuilt      1460 non-null   int64
 5   YearRemodAdd   1460 non-null   int64
 6   BsmtFinSF1     1460 non-null   int64
 7   BsmtFinSF2     1460 non-null   int64
 8   BsmtUnfSF      1460 non-null   int64
 9   TotalBsmtSF    1460 non-null   int64
 10  1stFlrSF       1460 non-null   int64
 11  2ndFlrSF       1460 non-null   int64
 12  LowQualFinSF   1460 non-null   int64
 13  GrLivArea      1460 non-null   int64
 14  BsmtFullBath   1460 non-null   int64
 15  BsmtHalfBath   1460 non-null   int64
 16  FullBath       1460 non-null   int64
 17  HalfBath       1460 non-null   int64
 18  BedroomAbvGr   1460 non-null   int64
 19  Kitche

In [ ]:
memory_reduction(nums)

In [ ]:
nums.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 34 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   MSSubClass     1460 non-null   int16
 1   LotArea        1460 non-null   int32
 2   OverallQual    1460 non-null   int8 
 3   OverallCond    1460 non-null   int8 
 4   YearBuilt      1460 non-null   int16
 5   YearRemodAdd   1460 non-null   int16
 6   BsmtFinSF1     1460 non-null   int16
 7   BsmtFinSF2     1460 non-null   int16
 8   BsmtUnfSF      1460 non-null   int16
 9   TotalBsmtSF    1460 non-null   int16
 10  1stFlrSF       1460 non-null   int16
 11  2ndFlrSF       1460 non-null   int16
 12  LowQualFinSF   1460 non-null   int16
 13  GrLivArea      1460 non-null   int16
 14  BsmtFullBath   1460 non-null   int8 
 15  BsmtHalfBath   1460 non-null   int8 
 16  FullBath       1460 non-null   int8 
 17  HalfBath       1460 non-null   int8 
 18  BedroomAbvGr   1460 non-null   int8 
 19  Kitche

In [ ]:
floats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MasVnrArea   1460 non-null   float64
 1   GarageYrBlt  1460 non-null   float64
dtypes: float64(2)
memory usage: 34.2 KB


In [ ]:
memory_reduction(floats)

In [ ]:
floats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MasVnrArea   1460 non-null   float16
 1   GarageYrBlt  1460 non-null   float16
dtypes: float16(2)
memory usage: 17.1 KB


In [ ]:
labels.info()

<class 'pandas.core.series.Series'>
Int64Index: 1460 entries, 1 to 1460
Series name: SalePrice
Non-Null Count  Dtype
--------------  -----
1460 non-null   int64
dtypes: int64(1)
memory usage: 55.1 KB


In [ ]:
memory_reduction

<function __main__.memory_reduction(df)>

In [ ]:
train_data_cleaned = cats.merge(nums, left_index=True, right_index=True).merge(floats, left_index=True, right_index=True)

In [ ]:
train_data_cleaned

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice,MasVnrArea,GarageYrBlt
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,2,2008,208500,196.0,2003.0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,5,2007,181500,0.0,1976.0
3,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,9,2008,223500,162.0,2001.0
4,0,0,0,1,0,0,1,1,0,0,...,272,0,0,0,0,2,2006,140000,0.0,1998.0
5,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,12,2008,250000,350.0,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,8,2007,175000,0.0,1999.0
1457,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,2,2010,210000,119.0,1978.0
1458,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,2500,5,2010,266500,0.0,1941.0


In [ ]:
train_data_cleaned.to_csv(output+'train_data_cleaned.csv')